In [453]:
# Code to check if left or right mouse buttons were pressed
import win32api
from time import sleep
import pyautogui as mouse
#import tkinter as tk
from tkinter import *
from tkinter import messagebox
import threading
from datetime import datetime
import keyboard  # using module keyboard
import pythoncom, pyHook

In [454]:
left_click_text = "Left click "
right_click_text = "Right click "
double_prefix = "Double "
delay_text = "Delay "
double_click_delay = 200

In [455]:
record_toggle = 0
play_toggle = 0

is_shift = False
is_ctrl = False
is_alt = False

is_recording = False
is_running = False
last_click = datetime.now()

In [456]:
def getText():
    textField.configure(state="normal")
    s = textField.get("1.0", END)
    textField.configure(state="disabled")    
    return s

In [457]:
def addText(text):
    textField.configure(state="normal")
    textField.insert(END, text + '\n')
    textField.configure(state="disabled")    

In [458]:
def deleteText():
    textField.configure(state="normal")
    textField.delete("1.0", END)
    textField.configure(state="disabled")    

In [459]:
def getDelay():
    global last_click
    return int((datetime.now() - last_click).total_seconds() * 1000)

In [460]:
def removeFromEnd(count):
    s = getText()
    deleteText()
    addText(s[:len(s) - count])

In [461]:
def removeLastLine():
    s = textField.get("1.0", END)
    removeFromEnd(len(s) - s.rfind('\n', 0, len(s)-5))

In [462]:
def addDelay():
    global last_click
    delay = getDelay()
    
    addText(delay_text + str(delay))
        
    last_click = datetime.now()
    return delay

In [463]:
def record(): 
    global is_recording
    if is_recording:
        is_recording = False
        return
    
    thread = threading.Thread(target=record_thread)
    thread.start()

def record_thread():
    global is_recording
    global last_click
    is_recording = True
    

    textField.configure(state="disabled")
    state_left = win32api.GetKeyState(0x01)  # Left button down = 0 or 1. Button up = -127 or -128
    state_right = win32api.GetKeyState(0x02)  # Right button down = 0 or 1. Button up = -127 or -128
    
    last_click = datetime.now()
    
    while is_recording:
        a = win32api.GetKeyState(0x01)
        b = win32api.GetKeyState(0x02)

        if a != state_left or b != state_right:
            pos = mouse.position()
            text = ""
                        
            if a != state_left:  # Button state changed
                state_left = a
                if a < 0:
                    text += left_click_text
                else:
                    continue
                
            if b != state_right:  # Button state changed
                state_right = b
                if b < 0:
                    text += right_click_text
                else:
                    continue
                    
            click_delay = addDelay()
            if click_delay < double_click_delay:
                removeLastLine()
                removeLastLine()
                text = double_prefix + text
                   
            text += str(pos.x) + ", " + str(pos.y)
            
            addText(text)
            
    removeLastLine()
    removeLastLine()
    
    textField.configure(state="normal")

In [464]:
def parse(_line):
    line = _line.strip()
    clicks = 1
    if line.startswith(double_prefix):
        clicks = 2
        line = line.replace(double_prefix, "")
    
    leftRight = getLeftRight(line)
    if leftRight != None:
        mouse.click(button=leftRight[0], clicks=clicks, x=leftRight[1], y=leftRight[2])
        return
    elif line.startswith(delay_text):
        line = line.replace(delay_text, "")
        delay = int(line) / 1000
        sleep(delay)      

In [465]:
def getLeftRight(line):
    s = line
    if line.startswith(left_click_text):
        s = line[len(left_click_text):len(line)]
        side = "left"
    elif line.startswith(right_click_text):
        s = line[len(right_click_text):len(line)]
        side = "right"
    
    if len(s) != len(line):
        p = s.split(", ")
        return (side, int(p[0]), int(p[1]))
    return None

In [466]:
def play():
    global is_running
    if is_running:
        is_running = False
        return
    
    thread = threading.Thread(target=play_thread)
    thread.start()

def play_thread():
    global is_running
    is_running = True
    lines = textField.get("1.0", END).splitlines()
        
    while is_running:
        for line in lines:
            if not is_running:
                return
            parse(line)    

In [467]:
def toggleButton(toggle, action=None):
    if action != None:
        action()
        
    if toggle.config('relief')[-1] == 'sunken':
        toggle.config(relief="raised")
    else:
        toggle.config(relief="sunken")

In [468]:
def createToggle(text, side, action):
    toggle = Button(topButtons, text=text, relief="raised", command=lambda: toggleButton(toggle, action))
    toggle.pack(side=side, pady=5, padx=5)
    return toggle

In [469]:
def addTextField():
    global textField
    scrollbar = Scrollbar(root)
    textField = Text(root, height=4, width=10000)
    scrollbar.pack(side=RIGHT, fill=Y)
    textField.pack(side=LEFT, fill=Y)
    scrollbar.config(command=textField.yview)
    textField.config(yscrollcommand=scrollbar.set)  
    #textField.bind('<F7>', onStopHotkey)

In [470]:
def addTopButtons():
    global topButtons
    topButtons = Frame(root)    
    topButtons.pack()

In [471]:
def on_closing():
    global is_running
    global is_recording
    is_recording = False
    is_running = False    
    pythoncom.PumpWaitingMessages()   
    root.destroy()

In [472]:
def onStopHotkey(event):
    global is_running
    global play_toggle
    global is_recording
    global record_toggle
    
    is_running = False
    is_recording = False
     
    for toggle in play_toggle, record_toggle:
        if toggle.config('relief')[-1] == 'sunken':
            toggle.config(relief="raised")

In [473]:
def onKeyboardUpEvent(event):
    global is_shift
    global is_ctrl
    global is_alt
    
    if is_recording or is_running: 
        key = str(event.GetKey())
        
        if "control" in key:
            is_ctrl = False
        elif "menu" in key:
            is_alt = False
        elif "shift" in key:
            is_shift = False
            
    return True

In [474]:
def onKeyboardDownEvent(event):
    global is_recording
    global textField
    global is_shift
    global is_ctrl
    global is_alt

    if is_recording or is_running:
        
        key = str(event.GetKey())
        
        text = ""
        if is_ctrl:
            text += "strg + "
        if is_alt:
            text += "alt + "
        if is_shift:
            text += "shift + "
        
        
        if "control" in key:
            is_ctrl = True
            return True
        elif "menu" in key:
            is_alt = True
            return True
        elif "shift" in key:
            is_shift = True
            return True
        
        click_delay = addDelay()
        addText(text + key)
        
        if str(event.GetKey()) == "F7":
            onStopHotkey(event)
    return True

In [475]:
root = Tk()
root.title("Auto Clicker")

addTopButtons()
addTextField()

record_toggle = createToggle("Record", LEFT, record)
play_toggle = createToggle("Play", RIGHT, play)

hm = pyHook.HookManager()
hm.KeyDown = onKeyboardDownEvent
hm.KeyUp = onKeyboardUpEvent
hm.HookKeyboard()

root.geometry("500x500")
root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()

Toggle Play
Toggle Record
Finished toggling
Toggle Play
Toggle Record
Finished toggling
